In [117]:
pip install furl

You should consider upgrading via the '/Users/mingtayang/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import requests
import bs4
import urllib.parse
import time
import random
import re
from furl import furl

# Define Variables

In [2]:
# Define keywords
keyword_primary = "iOS"
keyword_secondary_list = ['遠端', '遠距', '在家']

In [3]:
# Define original url
url_original = 'https://www.104.com.tw/jobs/search/?ro=0&jobcat=2007001004&kwop=7&keyword=%E9%81%A0%E7%AB%AF&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=14&asc=0&page=1&mode=s&jobsource=104_bank1'

# Gather web page results

In [4]:
# Define how to tell a page is invalid
def is_invalid_page(content):
    select = content.find('p', class_='b-tit')
    return True if select else False

In [5]:
f = furl(url_original)

In [6]:
f.args['keyword'] = keyword_primary 

In [7]:
page_soups = []
page_count = 1
while True:
    print(f'Start fetching page {page_count}...')
    f.args['page'] = page_count
    print(f.url)
    
    r = requests.get(f.url) 
    if r.status_code != 200:
        break
    
    soup = bs4.BeautifulSoup(r.text, 'lxml')
    if is_invalid_page(soup):
        break
        
    page_soups.append(soup)
    time.sleep(random.randint(3, 5))
    page_count += 1

Start fetching page 1...
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2007001004&kwop=7&keyword=iOS&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=14&asc=0&page=1&mode=s&jobsource=104_bank1
Start fetching page 2...
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2007001004&kwop=7&keyword=iOS&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=14&asc=0&page=2&mode=s&jobsource=104_bank1
Start fetching page 3...
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2007001004&kwop=7&keyword=iOS&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=14&asc=0&page=3&mode=s&jobsource=104_bank1
Start fetching page 4...
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2007001004&kwop=7&keyword=iOS&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=14&asc=0&page=4&mode=s&jobsource=104_bank1
Start fetching page 5...
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2007001004&kwop=7&keyword=iOS&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=14&asc=0&page=5&mode=s&jobsource=

Start fetching page 42...
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2007001004&kwop=7&keyword=iOS&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=14&asc=0&page=42&mode=s&jobsource=104_bank1
Start fetching page 43...
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2007001004&kwop=7&keyword=iOS&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=14&asc=0&page=43&mode=s&jobsource=104_bank1
Start fetching page 44...
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2007001004&kwop=7&keyword=iOS&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=14&asc=0&page=44&mode=s&jobsource=104_bank1
Start fetching page 45...
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2007001004&kwop=7&keyword=iOS&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=14&asc=0&page=45&mode=s&jobsource=104_bank1
Start fetching page 46...
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2007001004&kwop=7&keyword=iOS&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=14&asc=0&page=46&mode=s&

In [8]:
print(f'Found {len(page_soups)} Pages.')

Found 63 Pages.


# Use secondary keywords to filter the results

In [9]:
regex = '|'.join(keyword_secondary_list)
flags = re.IGNORECASE

In [11]:
results = []
job_urls = []
for soup in page_soups:
    articles = soup.find_all('article')
    job_urls.extend([urllib.parse.urljoin(url_original, article.find('a')['href']) for article in articles if article.find('a')])
print(f'Totally {len(job_urls)} jobs to filter.')
    
count = 0
for url in job_urls:
    r = requests.get(url)
    print(f'Start fetching url: {url}...')

    if re.search(regex, r.text.replace('ios-app', 'app'), flags = re.IGNORECASE):
        results.append(url)

    count += 1
    print(f'Filter progress: {round(count/len(job_urls)*100, 2)}%')

    time.sleep(random.randint(3, 5))


Totally 1251 jobs to filter.
Start fetching url: https://www.104.com.tw/job/6ztl0?jobsource=hotjob_chr...
Filter progress: 0.08%
Start fetching url: https://www.104.com.tw/job/7aabe?jobsource=hotjob_chr...
Filter progress: 0.16%
Start fetching url: https://www.104.com.tw/job/73ppk?jobsource=jolist_b_relevance...
Filter progress: 0.24%


KeyboardInterrupt: 

In [ ]:
print(f'Found {len(results)} results with the primary keyword "{keyword_primary}" and the secondary keyword "{keyword_secondary_list}".')

# Show the results

In [ ]:
for result in results:
    print(result)